## Cálculo votos

Nesse notebook são calculados os votos dos parlamentares de São Paulo

In [8]:
import pandas as pd
from utils.load_csv import load_csv
from utils.save_csv import save_csv

In [9]:
deputados_eleitos = load_csv('deputados_eleitos.csv')

In [10]:
rmsp = ["ARUJÁ", "BARUERI", "BIRITIBA MIRIM", "CAIEIRAS", "CAJAMAR", "CARAPICUÍBA", "COTIA", "DIADEMA", "EMBU DAS ARTES", "EMBU-GUAÇU", "FERRAZ DE VASCONCELOS", "FRANCISCO MORATO",
        "FRANCO DA ROCHA", "GUARAREMA", "GUARULHOS", "ITAPECERICA DA SERRA", "ITAPEVI", "ITAQUAQUECETUBA", "JANDIRA", "JUQUITIBA", "MAIRIPORÃ", "MAUÁ", "MOGI DAS CRUZES", "OSASCO",
        "PIRAPORA DO BOM JESUS", "POÁ", "RIBEIRÃO PIRES", "RIO GRANDE DA SERRA", "SALESÓPOLIS", "SANTA ISABEL", "SANTANA DE PARNAÍBA", "SANTO ANDRÉ", "SÃO BERNARDO DO CAMPO",
        "SÃO CAETANO DO SUL", "SÃO LOURENÇO DA SERRA", "SÃO PAULO", "SUZANO", "TABOÃO DA SERRA", "VARGEM GRANDE PAULISTA"]

In [11]:
votos_total = deputados_eleitos.groupby(["NM_CANDIDATO"])["QT_VOTOS_NOMINAIS"].sum().rename("votos_total")

votos_capital = deputados_eleitos[deputados_eleitos["NM_MUNICIPIO"] == "SÃO PAULO"].groupby("NM_CANDIDATO")["QT_VOTOS_NOMINAIS"].sum().rename("votos_capital")

votos_rmsp = deputados_eleitos[deputados_eleitos["NM_MUNICIPIO"].isin(rmsp)].groupby("NM_CANDIDATO")["QT_VOTOS_NOMINAIS"].sum().rename("votos_rmsp")

votos_interior = (deputados_eleitos[~deputados_eleitos["NM_MUNICIPIO"].isin(rmsp)].groupby("NM_CANDIDATO")["QT_VOTOS_NOMINAIS"].sum().rename("votos_interior"))

votos = pd.concat([votos_total, votos_capital, votos_rmsp, votos_interior], axis = 1).fillna(0)

In [12]:
dados = []

for index, row in deputados_eleitos.drop_duplicates(subset = 'NM_CANDIDATO').iterrows():

    nome = row['NM_CANDIDATO']

    parlamentar = {
        'id_deputado': row['SQ_CANDIDATO'],
        'nome_civil': row['NM_CANDIDATO'],
        'nome_urna': row['NM_URNA_CANDIDATO'],
        #'cpf': row['cpf'],
        #'nascimento': row['nascimento'],
        'eleicao': row['DS_ELEICAO'],
        'cargo': row['DS_CARGO'],
        'partido': row['NM_PARTIDO'],
        'votos_total': int(votos.loc[nome, 'votos_total']) if nome in votos.index else 0,
        'votos_capital': int(votos.loc[nome, 'votos_capital']) if nome in votos.index else 0,
        'votos_rmsp': int(votos.loc[nome, 'votos_rmsp']) if nome in votos.index else 0,
        'votos_interior': int(votos.loc[nome, 'votos_interior']) if nome in votos.index else 0,
        'rep_capital': (int(votos.loc[nome, 'votos_capital']) / int(votos.loc[nome, 'votos_total']) * 100) if (nome in votos.index and votos.loc[nome, 'votos_total'] > 0) else 0.0,
        'rep_rmsp': (int(votos.loc[nome, 'votos_rmsp']) / int(votos.loc[nome, 'votos_total']) * 100) if (nome in votos.index and votos.loc[nome, 'votos_total'] > 0) else 0.0,
        'rep_interior': (int(votos.loc[nome, 'votos_interior']) / int(votos.loc[nome, 'votos_total']) * 100) if (nome in votos.index and votos.loc[nome, 'votos_total'] > 0) else 0.0,
        'ranking_total': votos['votos_total'].sort_values(ascending = False).reset_index().index[votos['votos_total'].sort_values(ascending = False).reset_index()['NM_CANDIDATO'] == nome][0] + 1 if nome in votos.index else None,
        'ranking_capital': votos['votos_capital'].sort_values(ascending = False).reset_index().index[votos['votos_capital'].sort_values(ascending = False).reset_index()['NM_CANDIDATO'] == nome][0] + 1 if nome in votos.index else None,
        'ranking_rmsp': votos['votos_rmsp'].sort_values(ascending = False).reset_index().index[votos['votos_rmsp'].sort_values(ascending = False).reset_index()['NM_CANDIDATO'] == nome][0] + 1 if nome in votos.index else None,
        'ranking_interior': votos['votos_interior'].sort_values(ascending = False).reset_index().index[votos['votos_interior'].sort_values(ascending = False).reset_index()['NM_CANDIDATO'] == nome][0] + 1 if nome in votos.index else None
    }

    dados.append(parlamentar)

dados_parlamentares = pd.DataFrame(dados)

In [13]:
dados_parlamentares

,id_deputado,nome_civil,nome_urna,eleicao,cargo,partido,votos_total,votos_capital,votos_rmsp,votos_interior,rep_capital,rep_rmsp,rep_interior,ranking_total,ranking_capital,ranking_rmsp,ranking_interior
0,250001611324,MARIO LUIS FRIAS,MARIO FRIAS,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Federal,Partido Liberal,122564,37048,59916,62648,30.227473,48.885480,51.114520,81,69,85,82
1,250001611351,PAULO ROBERTO FREIRE DA COSTA,PAULO FREIRE DA COSTA,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Federal,Partido Liberal,161675,37503,70286,91389,23.196536,43.473635,56.526365,51,68,71,48
2,250001610654,JILMAR AUGUSTINHO TATTO,JILMAR TATTO,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Federal,Partido dos Trabalhadores,157843,67148,101416,56427,42.541006,64.251186,35.748814,52,37,49,92
3,250001610674,CARLOS ALBERTO ROLIM ZARATTINI,CARLOS ZARATTINI,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Federal,Partido dos Trabalhadores,147349,51854,88740,58609,35.191281,60.224365,39.775635,61,49,59,89
4,250001604967,RICARDO AUGUSTO MACHADO DA SILVA,RICARDO SILVA,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Federal,Partido Social Democrático,133936,3930,6320,127616,2.934237,4.718672,95.281328,72,133,141,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,250001619401,GUILHERME DA COSTA AGUIAR CORTEZ,GUILHERME CORTEZ,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Estadual,Partido Socialismo e Liberdade,45094,12848,18270,26824,28.491595,40.515368,59.484632,164,106,123,138
160,250001619350,EDIANE MARIA DO NASCIMENTO,EDIANE MARIA,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Estadual,Partido Socialismo e Liberdade,175617,96603,130567,45050,55.007773,74.347586,25.652414,46,18,29,109
161,250001619328,PAULA NUNES DOS SANTOS,PAULA DA BANCADA FEMINISTA,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Estadual,Partido Socialismo e Liberdade,259771,137754,187431,72340,53.029014,72.152396,27.847604,11,10,10,69
162,250001619379,MÔNICA CRISTINA SEIXAS BONFIM,MONICA DO MOVIMENTO PRETAS,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Estadual,Partido Socialismo e Liberdade,106781,51855,80584,26197,48.562010,75.466609,24.533391,96,48,62,140


In [14]:
save_csv(dados_parlamentares, 'dados_parlamentares.csv')

Base salva em data\dados_parlamentares.csv
